# 🍎🍅🍑

## <span style="color:#FFC1C1; font-weight:bold">1. 텐서 (Tensors) 실습</span>

앞서 배운 텐서의 기본적인 연산들을 이용해서 밑바닥에서 모델을 만들어보겠습니다! 

'사과, 토마토, 복숭아'를 분류하는 모델을 만든다고 해볼게요 🍎🍅🍑

In [1]:
import torch

### 01. 이미지 텐서 초기화 

이미지를 표현하는 3D 텐서를 만들어볼게요. (예. 높이 28, 너비 28, 채널 3)

In [2]:
image_tensor = torch.rand((28, 28, 3))
image_tensor.shape

torch.Size([28, 28, 3])

### 02. 미니 배치 텐서 만들기 

딥러닝 모델을 학습시킬때 이미지를 하나씩 모델에 입력해주면, 연산이 과도하게 많아집니다. 

보통 여러개의 이미지를 묶은 미니 배치를 만들어서 배치 단위로 모델을 학습시키죠. 

마찬가지로 여러 개의 이미지를 랜덤하게 생성해서 배치 단위로 묶어보겠습니다~ 

In [11]:
# 32개의 이미지를 랜덤하게 생성하고 배치로 묶기
batch_images = torch.stack([torch.rand((28, 28, 3)) for _ in range(32)], dim=0)

### 03. 미니 배치 텐서의 모양 확인 후 텐서 차원 조작하기 

배치 단위의 텐서의 모양을 확인해봅시다~ 

직접 텐서의 '모양'을 확인하는 코드를 작성해주세요! 

In [12]:
# print()
batch_images.shape

torch.Size([32, 28, 28, 3])

앗! torch.Size([32, 28, 28, 3])가 나왔나요? 


하지만 이미지를 넣어줄 때 보통 (B, C, H, W)로 맞춰준답니다~ 


그렇다면 여기서 어떻게 텐서의 차원을 조작할 수 있을까요? 

In [13]:
# hint : permute 
batch_images = batch_images.permute(0, 3, 1, 2)
batch_images.shape

torch.Size([32, 3, 28, 28])

다시 텐서의 모양을 확인해봅시다! 

In [14]:
# print()


### 04. W, b와 같은 파라미터 텐서 만들기 

딥러닝 모델은 행렬 연산이 전부입니다! 

데이터 행렬과 Weights, Bias와 같은 파라미터 행렬을 행렬 곱해주는 것이죠! 

이때, requires_grad 속성을 True로 설정하는걸 잊으면 안되겠죠~? 

또한, 신경망에서는 입력 이미지를 1차원으로 펼치고 가중치 행렬과의 행렬 곱 연산을 통해 모델을 학습시키는 것이 일반적입니다. (flattening) 

출력 클래스는 '사과, 토마토, 복숭아'로 3개니까 이에 맞춰서 파라미터를 초기화해볼게요~ 

In [20]:
# w는 임의의 값, b는 0으로 초기화
# w = 
# b = 
w = torch.randn((3*28*28, 3), requires_grad=True) # (channel*width*hight), output_class
b = torch.zeros(3, requires_grad=True)



In [21]:
# w와 b의 값 확인
print("w: ", w.shape)
print("b: ", b.shape)

w:  torch.Size([2352, 3])
b:  torch.Size([3])


### 05. 행렬 연산을 통해 output 만들기 

행렬 연산이 이루어지려면 이미지를 1차원 텐서로 펴서 연산을 수행해야겠죠? 

어떻게 텐서를 펼칠 수 있을까요? 

In [22]:
# hint : 이미지를 '다시' 셰이핑해봅시다~
batch_images = batch_images.reshape(32, -1) #
print(batch_images.shape)
output = torch.matmul(batch_images, w) + b
output.shape


torch.Size([32, 2352])


torch.Size([32, 3])

### 06. Optimizer에 넣어 Backward 연산하기 

우선 optimizer를 설정합니다. 

친절한 torch는 'torch.optim' 라이브러리에 다양한 optimizer 클래스를 제공해줍니다~ 

가장 간단한 SGD Optimizer를 이용해볼까요? 

In [23]:
# SGD Optimizer 
optimizer = torch.optim.SGD([w,b], lr=0.01)

optimizer가 model을 훈련시키기 위해서는 정답 label이 있어야겠죠?  


정답 Label을 바탕으로 loss를 측정할 수 있을테니까요.


임의로 target을 만들어봅시다~ 이때 target은 32개의 이미지 각각에 대한 label을 포함한 1차원 텐서면 충분합니다! 

In [25]:
target = torch.randint(0, 3, (32, ))
print(target)

tensor([2, 1, 1, 0, 1, 2, 2, 0, 2, 1, 0, 2, 0, 0, 2, 1, 0, 0, 1, 1, 1, 2, 2, 2,
        1, 2, 2, 0, 2, 1, 2, 0])


이제 output과 target이 있으니 이를 바탕으로 cross-entropy를 계산해보겠습니다. 

torch.nn.functional 모듈은 다양한 loss를 제공해줍니다~


그 중에서 cross_entropy를 이용할게요. 

In [26]:
# Cross-Entropy Loss 계산
loss = torch.nn.functional.cross_entropy(output, target)
print(loss)

tensor(28.8612, grad_fn=<NllLossBackward0>)


이제 loss를 바탕으로 loss에 대한 gradient를 구해서 파라미터를 업데이트해줘야겠죠. 

backward() 메소드는 한번에 이러한 역전파 과정을 가능하게 합니다. 

In [27]:

loss.backward()

역전파시켜 얻은 gradient를 바탕으로 optimizer는 파라미터를 업데이트합니다. 

In [28]:
# Optimizer로 파라미터 업데이트
optimizer.step()

바뀐 w, b 파라미터 값을 확인해볼까요? 

In [29]:
# w와 b의 값 확인
print("w: ", w)
print("b: ", b)

w:  tensor([[-0.3163, -0.2173, -0.9719],
        [-0.1772,  0.1341, -2.0133],
        [-0.6357, -0.2038, -1.2570],
        ...,
        [-1.6403,  0.2311, -0.7192],
        [ 0.6805, -1.6807, -0.1569],
        [ 1.3096, -1.3393, -0.7055]], requires_grad=True)
b:  tensor([-0.0066,  0.0031,  0.0035], requires_grad=True)


그리고 이후의 학습을 위해 다시 optimizer는 gradient 행렬을 초기화합니다

In [38]:
# Gradient 초기화
optimizer.zero_grad()

이런 식으로 모델은 iteration을 반복하며 학습을 진행합니다. 

하지만, 안타깝게도 우리의 모델은 이렇게 간단하지는 않아요...

layer는 수없이 많은 파라미터로 이루어져있고, 이러한 layer가 모여 하나의 module을 만들고, module들이 모여 큰 model을 만들거든요. 

즉, 하나의 model은 수많은 Layer들로 이루어졌겠죠. 

한번 모델을 학습시키기 위해 일일이 Layer를 구성하는 parameter 텐서를 정의하고, optimizer에 정의한 각각의 layer를 넣어주는건 생각만해도 너무 복잡합니다. 

다행히 파이토치는 귀찮은 우리를 대신해서 레고 블럭 쌓듯 뚝딱! 하고 모델을 쌓을 수 있도록 하는 라이브러리를 제공합니다. 이 layer들은 순전파, 역전파 등이 기본으로 구현되어 있어 모델 훈련 및 활용이 매우 쉽습니다.